In [1]:
import pickle
from collections import OrderedDict
from pathlib import Path

import numpy as np

In [2]:
import elm

Using database version X4-2024-12-31 located in: /home/beyerk/db/exfor/unpack_exfor-2024/X4-2024-12-31


In [3]:
with open(Path("../prior/prior_distribution.pickle"), "rb") as f:
    prior = pickle.load(f)

In [4]:
corpus_dir = Path("../data/corpus/")
output_dir = Path("./")

In [5]:
with open(corpus_dir / "nn_elastic_data.pkl", "rb") as f:
    nn_elastic_data = pickle.load(f)
with open(corpus_dir / "pp_elastic_data.pkl", "rb") as f:
    pp_elastic_data = pickle.load(f)

In [6]:
with open(corpus_dir / "nn_elastic_workspaces.pkl", "rb") as f:
    nn_elastic_workspaces = pickle.load(f)
with open(corpus_dir / "pp_elastic_workspaces.pkl", "rb") as f:
    pp_elastic_workspaces = pickle.load(f)

In [7]:
nn_corpus = elm.corpus.ElasticAngularCorpus(
    elm.elm.calculate_elastic_differential_xs,
    "ELM",
    "(n,n)",
    elm.elm.NUM_PARAMS,
    "dXS/dA",
    nn_elastic_workspaces,
    nn_elastic_data,
)
pp_corpus = elm.corpus.ElasticAngularCorpus(
    elm.elm.calculate_elastic_differential_xs,
    "ELM",
    "(p,p)",
    elm.elm.NUM_PARAMS,
    "dXS/dRuth",
    pp_elastic_workspaces,
    pp_elastic_data,
)

In [8]:
combined_elastic_corpus = elm.corpus.Corpus(
    [nn_corpus, pp_corpus],
    elm.elm.NUM_PARAMS,
    "ELM",
    "elastic",
)

In [9]:
with open(output_dir / "nn_corpus.pkl", "wb") as f:
    pickle.dump(nn_corpus, f)
with open(output_dir / "pp_corpus.pkl", "wb") as f:
    pickle.dump(pp_corpus, f)
with open(output_dir / "combined_elastic_corpus.pkl", "wb") as f:
    pickle.dump(combined_elastic_corpus, f)

### This will give us an order of magnitude for how long a single sample in the calibration will take:

In [10]:
def to_ordered_dict(sample: np.ndarray):
    return OrderedDict(zip([p.name for p in elm.elm.params], sample))

In [11]:
%%timeit
nn_corpus.logpdf(to_ordered_dict(prior.mean))

143 ms ± 4.32 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit
pp_corpus.logpdf(to_ordered_dict(prior.mean))

357 ms ± 13.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%%timeit
combined_elastic_corpus.logpdf(to_ordered_dict(prior.mean))

495 ms ± 16.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## and just to make sure:

In [14]:
Ltot = combined_elastic_corpus.logpdf(to_ordered_dict(prior.mean)) 

In [15]:
Lnn = nn_corpus.logpdf(to_ordered_dict(prior.mean))

In [16]:
Lpp = pp_corpus.logpdf(to_ordered_dict(prior.mean))

In [17]:
np.testing.assert_almost_equal(Ltot , Lnn + Lpp)